In [243]:
#Import all the libraries we need
import os
import json
import pandas as pd

#Get a list of all the folders. The name of the folder is also the id number for the data package. Each folder
#contains the text of the pdf as a txt file and the json result from GNRD.
packages = []
annotation_file = open('dryad_results.txt', 'r')
next(annotation_file)
for line in annotation_file:
    row = line.split('\t')
    data_package = row[1]
    if data_package in packages:
        pass
    else:
        packages.append(data_package)
#packages = next(os.walk('pdf_text'))[1]
packages.remove('133')
print(len(packages))
'110' in packages

219


True

In [244]:
#Iterate over all the folders and grab the json results from each. From the json pick out the verbatim string that
#GNRD found and the "odds". The higher the odds the more likely the string is an actual name.
gnrd_results = {}
tot_words = {}
for p in packages:
    gnrd_names = []
    if p != '133':
        [data_file] = [x for x in os.listdir('pdf_text/' + p) if x.endswith(".json")]
        data = open('pdf_text/' + p + '/' + data_file,'r')
        result = json.load(data)
        metadata = result['metadata']
        total_words = metadata['total_words']
        names = result['names']
        if names == None:
            pass
        else:
            for n in names:
                string = n['verbatim']
                try:
                    odds = n['odds']
                except KeyError:
                    odds = 0
                gnrd_names.append(tuple([string,odds]))
        gnrd_results[p] = gnrd_names
        tot_words[p] = total_words
#print(gnrd_results['350'])

In [245]:
#Initialize the dictionary for the human annotator data
annotator_results = {}
for p in packages:
    annotator_results[p]=[]

In [246]:
#Read the file that contains the names found by the human annotators. Go over every row of the file and create a
#dictionary using the data package ID number as the key and a list of the name strings as the value.
# the name string is in a tuple with the type of string, i.e. scientific or vernacular name
annotation_file.seek(0)
next(annotation_file)
for line in annotation_file:
    row = line.split('\t')
    string = row[0]
    data_package = row[1]
    source_type = row[2]
    source = row[3]
    string_type = row[6]
    file_type = row[7]#this is wrong for the publication files
    if source_type == 'data' or source == 'GNRD_1':
        continue
    else:
        try:
            ls = annotator_results[data_package]
        except KeyError:
            print('possible missing data package')
            print(row)
        ls.append(tuple([string,string_type]))
        annotator_results[data_package] = ls
print('completed successfully')
#print(annotator_results['350'])

completed successfully


In [247]:
#Let's compare the list of names returned by GNRD with the list of names returned by the annotators
#for now, let's ignore the vernacular names in the annotator data

df = pd.DataFrame(columns=['data_package','true_positives','returned_results','annotator_results','precision',
                           'recall','F1','total_words'])

for p in packages:
    #grab the list for the data package from the gnrd results and the annotator results.
    #separate the name strings from the odds value in the gnrd list and the string type in the annotator list
    gnrd = gnrd_results[p]
    human = annotator_results[p]
    gnrd_string = []
    for n,s in gnrd:
        gnrd_string.append(n)
    gnrd_set = set(gnrd_string)
    human_string = []
    for n,s in human:
        if s != 'vernacular': #lets ignore vernacular names for the moment
            human_string.append(n)
    human_set = set(human_string)
    #now we have a set of name strings for gnrd and a set for the annotators
    #let's compare them
    g = len(gnrd_set) #total strings returned by GNRD
    h = len(human_set) #total strings returned by annotator
    overlap_set = gnrd_set.intersection(human_set) #this is the set of strings that are in the gnrd and annotator lists
    o = len(overlap_set)
    gnrd_only = gnrd_set.difference(human_set) #what is in gnrd list that is not in annotator list - false positives
    human_only = human_set.difference(gnrd_set) #what is in annotator list that is not in gnrd list - false negatives
    go = len(gnrd_only)
    ho = len(human_only)
    if go + o == g and ho + o == h: #adding a sanity check
        pass
    else:
        print('problem')
    if g == 0 or h == 0:
        precision = 'NULL'
        recall = 'NULL'
        F1 = 'NULL'
    else:
        precision = o / g
        recall = o / h
        if precision == 0 and recall == 0:
            F1 = 'NULL'
        else:
            F1 = 2*((precision*recall)/(precision+recall))
    #print(df)
    y = {'data_package':p,'true_positives':o,'returned_results':g,'annotator_results':h,
                    'precision':precision,'recall':recall,'F1':F1,'total_words':tot_words[p]}
    df = df.append(y,ignore_index=True)
print(df)

    data_package true_positives returned_results annotator_results precision  \
0              2            106              113               106  0.938053   
1              6              0                0                 0      NULL   
2             12              0                1                 0      NULL   
3             18              3               14                56  0.214286   
4             22            144              234               153  0.615385   
5             29            137              144               137  0.951389   
6             39             50               78                56  0.641026   
7             45             11               37                17  0.297297   
8             51             39               83               109   0.46988   
9             56              1                5                 2       0.2   
10            61              3                7                 3  0.428571   
11            67              0         

In [248]:
#Now we have a dataframe that contains performance data for each pdf 
#let's calculate performance overall
TOT = df.sum(axis=0)
print(TOT)
TP = TOT.iloc[1]
RR = TOT.iloc[2]
AR = TOT.iloc[3]
TW = TOT.iloc[4]
print(TP)
print(RR)
print(AR)
print(TW)

data_package         2612182229394551566167788488941001041101171211...
true_positives                                                    6367
returned_results                                                 11632
annotator_results                                                 9786
total_words                                                    1589064
dtype: object
6367
11632
9786
1589064


In [249]:
precision = TP / RR
recall = TP / AR
F1 = 2*((precision*recall)/(precision+recall))
print(F1)

0.5945466430105518
